In [8]:
# Logs: https://drive.google.com/drive/folders/1n-4bfiWS1LFQucPdNj0vhn_epwvmvB1V?usp=sharing
# Ficus version: 64d1ea7f74c9b5d28944aca8a9ccac75da23865c

import os.path

from ficus.analysis.patterns.patterns_models import *
from ficus.pipelines.analysis.analysis_parts import *
from ficus.pipelines.analysis.high_level import *
from ficus.pipelines.analysis.patterns.patterns_graph_parts import *
from ficus.pipelines.filtering.filter_parts import *
from ficus.pipelines.mutations.mutations_parts import *
from ficus.pipelines.serialization.pipeline_parts import *
from ficus.pipelines.start.start_parts import *

In [9]:
def procfiler_filter_predicate(event: MyEvent):
    return event[concept_name].startswith('Procfiler/')

event_class_first_open = '_{'
event_class_second_open = '/'

def first_class_extractor(name: str) -> str:
    if event_class_first_open not in name:
        return name

    return  name[:name.index(event_class_first_open)]

def strongest_class_extractor_evt(event: MyEvent) -> str:
    return first_class_extractor(event[concept_name])

def methods_filter_predicate(event: MyEvent):
    return event[concept_name].startswith('Method/')


def alloc_filter_predicate(event: MyEvent):
    return event[concept_name].startswith('GC/Sampled')

def activity_name_creator(result, ext):
    save_name = activity_name(result).replace('/', '_').replace('::', '_').replace('{', '_').replace('}', '_')
    if len(save_name) > 50:
        save_name = save_name[:50]

    return save_name + ext

def activity_node_name_creator(graph_node: GraphNode) -> str:
    return '\n'.join(graph_node.name.split('::'))

def activity_in_trace_filter(info: ActivityInTraceInfo) -> bool:
    if info.node.length == 1:
        return False

    return info.length > info.node.length / 2

def chaotic_events_filter(event: MyEvent):
    return 'GC/SampledObjectAllocation_{System.Int32[]}' in event[concept_name] or suspend_restart_ee_events_filter(event)


def suspend_restart_ee_events_filter(event: MyEvent):
    return 'RestartEE' in event[concept_name] or 'SuspendEE' in event[concept_name]


In [10]:
def silent_one_level_extracting_activities_pipeline():
    return Pipeline(
        DiscoverActivitiesFromTandemArrays(array_kind=TandemArrayKind.PrimitiveArray,
                                           activity_in_trace_filter=activity_in_trace_filter,
                                           activity_level=0),
        CreateLogFromActivities(),
        ClearActivities(),
        DiscoverActivitiesForSeveralLevels([default_class_extractor],
                                           discovering_strategy=ActivitiesDiscoveryStrategy.DiscoverFromAllTraces),
        CreateLogFromActivities(UndefinedActivityHandlingStrategy.DontInsert, use_hashes_as_names=False),
        ClearActivities(),
        DiscoverActivitiesInstancesFromRepeatsUntilNoMore(default_class_extractor,
                                                          use_hashes_as_event_name=False,
                                                          discovery_strategy=ActivitiesDiscoveryStrategy.DiscoverFromSingleMergedTrace),
        CreateLogFromActivities(use_hashes_as_names=False),
        PrintNumberOfUnderlyingEvents(),
    )

In [11]:
def analyze_consoleapp1_log():
    log_path = 'consoleapp1.xes'
    for with_methods in [True, False]:
        methods_folder = 'without_methods' if not with_methods else 'with_methods'
        file_name = os.path.splitext(log_path)[0]
        folder_path = os.path.join('results', file_name, methods_folder)
        
        def activity_save_path_creator(result, folder_name, ext):
            name = activity_name_creator(result, ext)
            return os.path.join(folder_path, folder_name, f'log_{name}')

        os.makedirs(folder_path, exist_ok=True)
        Pipeline(
            ReadLogFromXes(),
            RemoveEventsFromLogPredicate(procfiler_filter_predicate),
            TracesDiversityDiagram(plot_legend=False,
                                   save_path=os.path.join(folder_path, 'console_app1_initial_log.png')),
            RemoveEventsFromLogPredicate(chaotic_events_filter),
            TracesDiversityDiagram(plot_legend=False,
                                   save_path=os.path.join(folder_path, 'console_app1_after_filtering_log.png')),
            FilterTracesByCount(min_event_in_trace_count=3),
            RemoveEventsFromLogPredicate(methods_filter_predicate) if not with_methods else None,
            FilterTracesByVariants(),
            PrintEventLogInfo(),
            ParallelPipeline(
                silent_one_level_extracting_activities_pipeline(),
                Pipeline(
                    PrintEventLogInfoBeforeAndAfter('tandem_arrays', Pipeline(
                        DiscoverActivitiesFromTandemArrays(array_kind=TandemArrayKind.PrimitiveArray,
                                                           activity_in_trace_filter=activity_in_trace_filter,
                                                           activity_level=0),
                        DrawFullActivitiesDiagram(height_scale=50,
                                                  save_path=os.path.join(folder_path, 'console_app1_discovered_loops.png')),
                        CreateLogFromActivities(),
                    )),

                    ClearActivities(),

                    PrintEventLogInfoBeforeAndAfter('several_levels', Pipeline(
                        DiscoverActivitiesForSeveralLevels([default_class_extractor, strongest_class_extractor_evt],
                                                           discovering_strategy=ActivitiesDiscoveryStrategy.DiscoverFromAllTraces),
                        CalculatePercentageOfUnattachedEvents(),
                        DrawFullActivitiesDiagram(plot_legend=False,
                                                  height_scale=50,
                                                  save_path=os.path.join(folder_path, 'console_app1_full_activities_first_extraction.png')),
                        CreateLogFromActivities(UndefinedActivityHandlingStrategy.DontInsert, use_hashes_as_names=False),
                    )),

                    TracesDiversityDiagram(plot_legend=False,
                                           height_scale=50,
                                           save_path=os.path.join(folder_path, 'console_app1_after_several_levels_log.png')),
                    ClearActivities(),

                    PrintEventLogInfoBeforeAndAfter('final_repeats', Pipeline(
                        DiscoverActivitiesInstancesFromRepeatsUntilNoMore(default_class_extractor,
                                                                          use_hashes_as_event_name=False,
                                                                          discovery_strategy=ActivitiesDiscoveryStrategy.DiscoverFromAllTraces),
                        DrawFullActivitiesDiagram(plot_legend=False,
                                                  height_scale=50,
                                                  save_path=os.path.join(folder_path, 'console_app1_full_activities_second_extraction.png')),
                        CreateLogsForActivities(class_extractor=default_class_extractor, activity_level=0),

                        ExecuteWithEachActivityLog(Pipeline(
                            SubstituteUnderlyingEvents(),
                            TracesDiversityDiagram(plot_legend=True,
                                                   save_path=lambda x: activity_save_path_creator(x, 'activities_diagrams', '.png')),
                            RemoveEventsFromLogPredicate(lambda e: 'Method' in e[concept_name]),
                            IfPipeline(lambda ctx: len(log(ctx)) > 0, Pipeline(
                                ApplyClassExtractor(class_extractor=strongest_class_extractor_evt,
                                                    event_selector=lambda e: 'Assembly' in e[concept_name] or 'Loader' in e[concept_name]),
                                TracesDiversityDiagram(plot_legend=True,
                                                       save_path=lambda x: activity_save_path_creator(x, 'activities_diagrams', '_2.png')),
                                AddArtificialStartEndEvents(),
                                SaveEventLogToXes(save_path_creator=lambda x: activity_save_path_creator(x, 'activities_logs', '_2.xes'))
                            ))
                        )),

                        CreateLogFromActivities(use_hashes_as_names=False),
                    )),

                    TracesDiversityDiagram(plot_legend=False,
                                           height_scale=50,
                                           save_path=os.path.join(folder_path, 'console_app1_final_log.png')),
                    PrintNumberOfUnderlyingEvents(),
                )
            )
        )(log_path)

In [12]:
analyze_consoleapp1_log()

Number of traces in log: 1
Number of events in log: 675
Number of low-level event classes in log: 322
Discovered 8 activities
Discovered 9 activities instances
Discovered 31 activities
Discovered 221 activities instances
Discovered 29 activities
Discovered 63 activities instances
Discovered 23 activities
Discovered 49 activities instances
Discovered 22 activities
Discovered 48 activities instances
Underlying events count: 382
Operation tandem_arrays started
Number of traces in log: 1
Number of events in log: 675
Number of low-level event classes in log: 322
Discovered 8 activities
Discovered 9 activities instances
Number of traces in log: 1
Number of events in log: 646
Number of low-level event classes in log: 321
Operation tandem_arrays finished

Operation several_levels started
Number of traces in log: 1
Number of events in log: 646
Number of low-level event classes in log: 321
Discovered 31 activities
Discovered 221 activities instances
Discovered 4 activities
Discovered 409 activit

In [13]:
def analyze_array_pooling_log():
    log_path = 'array_pooling.xes'
    for with_methods in [True, False]:
        methods_folder = 'without_methods' if not with_methods else 'with_methods'
        file_name = os.path.splitext(log_path)[0]
        folder_path = os.path.join('results', file_name, methods_folder)
        
        height_scale = 10
        
        os.makedirs(folder_path, exist_ok=True)

        Pipeline(
            ReadLogFromXes(),
            RemoveEventsFromLogPredicate(procfiler_filter_predicate),
            TracesDiversityDiagram(plot_legend=False,
                                   height_scale=height_scale,
                                   save_path=os.path.join(folder_path, 'array_pooling_initial_log.png')),
            RemoveEventsFromLogPredicate(suspend_restart_ee_events_filter),
            TracesDiversityDiagram(plot_legend=False,
                                   height_scale=height_scale,
                                   save_path=os.path.join(folder_path, 'array_pooling_after_filtering_log.png')),
            FilterTracesByCount(min_event_in_trace_count=3),
            RemoveEventsFromLogPredicate(methods_filter_predicate) if not with_methods else None,
            FilterTracesByVariants(),
            PrintEventLogInfo(),
            ParallelPipeline(
                silent_one_level_extracting_activities_pipeline(),
                Pipeline(
                    PrintEventLogInfoBeforeAndAfter('tandem_arrays', Pipeline(
                        DiscoverActivitiesFromTandemArrays(array_kind=TandemArrayKind.PrimitiveArray,
                                                           activity_in_trace_filter=activity_in_trace_filter,
                                                           activity_level=0),
                        DrawFullActivitiesDiagram(height_scale=height_scale,
                                                  save_path=os.path.join(folder_path, 'array_pooling_discovered_loops.png')),

                        CreateLogFromActivities(),
                    )),

                    ClearActivities(),

                    PrintEventLogInfoBeforeAndAfter('several_levels', Pipeline(
                        DiscoverActivitiesForSeveralLevels([default_class_extractor, strongest_class_extractor_evt],
                                                           discovering_strategy=ActivitiesDiscoveryStrategy.DiscoverFromAllTraces),
                        CalculatePercentageOfUnattachedEvents(),
                        DrawFullActivitiesDiagram(plot_legend=False,
                                                  height_scale=height_scale,
                                                  save_path=os.path.join(folder_path, 'array_pooling_full_activities_first_extraction.png')),
                        CreateLogFromActivities(UndefinedActivityHandlingStrategy.DontInsert, use_hashes_as_names=False),
                    )),

                    TracesDiversityDiagram(plot_legend=False,
                                           save_path=os.path.join(folder_path, 'array_pooling_after_several_levels_log.png')),
                    ClearActivities(),

                    PrintEventLogInfoBeforeAndAfter('final_repeats', Pipeline(
                        DiscoverActivitiesInstancesFromRepeatsUntilNoMore(default_class_extractor,
                                                                          use_hashes_as_event_name=False,
                                                                          activity_in_trace_filter=activity_in_trace_filter,
                                                                          discovery_strategy=ActivitiesDiscoveryStrategy.DiscoverFromSingleMergedTrace),
                        DrawFullActivitiesDiagram(plot_legend=False,
                                                  width_scale=4,
                                                  save_path=os.path.join(folder_path, 'array_pooling_full_activities_second_extraction.png')),
                        CreateLogFromActivities(use_hashes_as_names=False),
                    )),

                    TracesDiversityDiagram(plot_legend=False,
                                           width_scale=5,
                                           height_scale=2,
                                           save_path=os.path.join(folder_path, 'array_pooling_final_log.png')),
                    PrintNumberOfUnderlyingEvents(),
                )
            )
        )(log_path)

In [14]:
analyze_array_pooling_log()

Number of traces in log: 50
Number of events in log: 163928
Number of low-level event classes in log: 237
Discovered 13 activities
Discovered 3780 activities instances
Discovered 69 activities
Discovered 11491 activities instances
Discovered 154 activities
Discovered 537 activities instances
Discovered 131 activities
Discovered 391 activities instances
Discovered 141 activities
Discovered 384 activities instances
Underlying events count: 151459
Operation tandem_arrays started
Number of traces in log: 50
Number of events in log: 163928
Number of low-level event classes in log: 237
Discovered 13 activities
Discovered 3780 activities instances
Number of traces in log: 50
Number of events in log: 59243
Number of low-level event classes in log: 244
Operation tandem_arrays finished

Operation several_levels started
Number of traces in log: 50
Number of events in log: 59243
Number of low-level event classes in log: 244
Discovered 69 activities
Discovered 11491 activities instances
Discovered 